In [10]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import dotenv
from google.api_core import retry
dotenv.load_dotenv()

import itertools
import os
from datasets import load_dataset

In [11]:
GEMINI_API_KEYs = os.getenv("GEMINI_API_KEY").strip().split(',')

GEMINI_API_KEYs = [key for key in GEMINI_API_KEYs if key]
keys = itertools.cycle(GEMINI_API_KEYs)
emb_model_name = "models/embedding-001"

In [13]:
@retry.Retry(timeout=30.0)
def get_embeddings(text):
    genai.configure(api_key=next(keys))
    embedding = genai.embed_content(model=emb_model_name,
                                    content=text,
                                    task_type="CLUSTERING")
    return embedding['embedding']

In [14]:
ds = load_dataset("nlplabtdtu/ppl_100k", split="train")

#create new column for embeddings
new_ds = ds.map(lambda x: {"embedding": get_embeddings(x["text"][:5000]), "text": x['text']}, num_proc=32)

Map (num_proc=32): 100%|██████████| 100000/100000 [34:17<00:00, 48.60 examples/s]


In [16]:
new_ds[0]

{'text': 'Du lịch Vũng Tàu: Quảng bá du lịch thông qua các sự kiện, hiệu quả và chất lượng ngành du lịch | du lịch Vũng Tàu\nSkip to content\nDu Lịch Vũng Tàu\nKeywords\nTin tức\nCà phê\nGiải trí\nKhách sạn\nTham quan\nẨm thực\nDanh mục >> Tin tức - Sự kiện\t\nQuảng bá du lịch thông qua các sự kiện, hiệu quả và chất lượng ngành du lịch\nGởi nhận xét Nhận xét\nChia sẻ Chia sẻ\nChia sẻ:\nBấm để chia sẻ trên Twitter (Opens in new window)\nNhấn vào chia sẻ trên Facebook (Opens in new window)\nBấm để chia sẻ trên Google+ (Opens in new window)\nTrong bối cảnh đó các tỉnh thành, quốc gia đều đang tập trung nguồn lực để quảng bá hình ảnh, đất nước, con người bằng hình thức trực tuyến để cung cấp thông tin, quảng bá, xúc tiến du lịch và chuẩn bị để thu hút khách khi có điều kiện, việc Vũng Tàu tổ chức các sự kiện để kích cầu sau đại dịch và vào mùa du lịch thấp điểm này như Hoa Hậu Việt Nam, các Giải đua Xe Đạp, Giải Bóng rổ mở rộng; hoặc Giải Bóng đá, trước đó Vũng Tàu vinh dự sở hữu nhiề

In [17]:
new_ds.push_to_hub("nlplabtdtu/ppl_100k_with_embeddings", token=os.getenv("HF_AUTH_TOKEN"))

Uploading the dataset shards: 100%|██████████| 5/5 [01:40<00:00, 20.18s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/nlplabtdtu/ppl_100k_with_embeddings/commit/30d2fb887d85ee8d8deb41ce7977c195223a7219', commit_message='Upload dataset', commit_description='', oid='30d2fb887d85ee8d8deb41ce7977c195223a7219', pr_url=None, pr_revision=None, pr_num=None)